<a href="https://colab.research.google.com/github/AROMAL2059/Pwc_Digital_Intelligence.ipynb/blob/main/Task_3_Quant_Finance_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


In [3]:
# Read the CSV file (using ';' as the separator)
data = pd.read_csv("/content/Data.csv", sep=";")

In [4]:
# Set parameters
n_forecast = 30  # number of forecast periods
n_vintage = data.shape[0]  # number of vintages
discount_rate = 0.025  # annual discount rate

In [5]:
# Extract the historical cash flows (from the 3rd column onward) and amounts originated (2nd column)
historical_cf = data.iloc[:, 2:].values  # converting to a NumPy array
amount_originated = data.iloc[:, 1].values  # 1D array

In [6]:

# Compute periods remaining per vintage (R's n.vintage:1 produces a descending sequence)
# In Python, we create a descending array from n_vintage down to 1, then subtract from n_forecast.
periods_remaining = n_forecast - np.arange(n_vintage, 0, -1)


In [7]:
# Compute the historical paid percentages: observed repayments divided by the originated amount (broadcasting row-wise)
# Note: historical_cf is a 2D array and amount_originated is a 1D array.
paid_percentages = historical_cf / amount_originated[:, None]

In [8]:
# Extract the first period repayment percentage (diagonal elements)
first_period = np.diag(paid_percentages)

In [9]:
# For the second period, take the diagonal of paid_percentages (excluding last row and first column),
# then append the last vintage's second period as twice its first period.
second_period_part = np.diag(paid_percentages[:-1, 1:])
second_period_last = paid_percentages[-1, -1] * 2  # bottom-right element * 2
second_period = np.concatenate((second_period_part, [second_period_last]))

In [10]:
# Initialize matrix p (n_vintage x n_forecast) with zeros
p = np.zeros((n_vintage, n_forecast))
p[:, 0] = first_period      # first period (index 0)
p[:, 1] = second_period     # second period (index 1)

In [11]:
# Compute expected repayment percentages for periods 3 to n_forecast
for i in range(n_vintage):
    for j in range(2, n_forecast):  # j=2 corresponds to the third period
        cumulative_sum = np.sum(p[i, :j])  # sum of p[i, 0] to p[i, j-1]
        p[i, j] = max(0, p[i, 1] * np.log(1 + (1 - cumulative_sum)) * (1 - j / n_forecast))


In [12]:
# Create matrix for forecasted repayment percentages (n_vintage x (n_forecast-1))
p_forecast = np.zeros((n_vintage, n_forecast - 1))
for i in range(n_vintage):
    # For vintage i, assign the expected repayment percentages to the future periods.
    # The correct starting column in p is: n_forecast - periods_remaining[i]
    start_index = n_forecast - periods_remaining[i]
    # Loop over the number of forecast periods for this vintage
    for j in range(periods_remaining[i]):
        p_forecast[i, j] = p[i, start_index + j]

In [13]:
# Compute discount factors for each future period (from 1 to n_forecast-1)
# Note: we convert the annual discount rate to a monthly factor.
discount_factors = 1 / ((1 + discount_rate) ** (np.arange(1, n_forecast) / 12))

In [14]:
# Compute the present value (PV) of forecasted cash flows.
# Multiply each forecasted percentage by the corresponding discount factor and the originated amount.
pv = p_forecast * discount_factors[None, :] * amount_originated[:, None]

# Sum all present values to get the portfolio value
result = pv.sum()


In [15]:
# Print the result rounded to two decimal places
print("The fair value estimate for the portfolio is", round(result, 2), "Swiss Francs")

The fair value estimate for the portfolio is 84779941.82 Swiss Francs
